<a href="https://colab.research.google.com/github/havihaviplants/Machine-learning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
# 자신의 Google Drive 마운트하는 코드를 추가하자!
mnist = np.load('/content/drive/MyDrive/MY Drive/mnist.npz')
x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [ ]:
class CNN(nn.Module):
 def __init__(self):
  super().__init__()
  self.cnn = nn.Sequential(
 nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1), # [b, 1, 28, 28] -> [b, 16, 28, 28]
 nn.ReLU(True),
 nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1), # [b, 16, 28, 28] -> [b, 16, 28, 28]
 nn.ReLU(True),
 nn.MaxPool2d(2, stride=2), # [b, 16, 28, 28] -> [b, 16, 14, 14]
 nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), # [b, 16, 14, 14] -> [b, 32, 14, 14]
 nn.ReLU(True),
 nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1), # [b, 32, 14, 14] -> [b, 32, 14, 14]
 nn.ReLU(True),
 nn.MaxPool2d(2, stride=2), # [b, 32, 14, 14] -> [b, 32, 7, 7]
 )
  self.fc1 = nn.Linear(32 * 7 * 7, 256)
  self.fc2 = nn.Linear(256, 10)

 def forward(self, x):
  x = self.cnn(x)
  x = torch.flatten(x, start_dim=1)
  x = F.relu(self.fc1(x))
  x = self.fc2(x)

  return x


In [ ]:
from torch.optim import Adam
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
opti = Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(model, dataloader, criterion, data_len, opti):
 correct = 0

 model.train()
 for data, target in dataloader:
  data = data.view(-1, 1, 28, 28).to(device)
  target = target.to(device)

  output = model(data)
  loss = criterion(output, target)

  opti.zero_grad()
  loss.backward()
  opti.step()

  pred = output.max(1, keepdim=True)[1]
  correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc


In [ ]:
def evaluate(model, dataloader, criterion, data_len):
 correct = 0

 model.eval()
 for data, target in dataloader:
  data = data.view(-1, 1, 28, 28).to(device)
  target = target.to(device)

  output = model(data)
  loss = criterion(output, target)

  pred = output.max(1, keepdim=True)[1]
  correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc


In [ ]:
epoch = 20

for i in range(epoch):
 train_acc = train(model, train_dataloader, criterion, len(train_dataloader.dataset), opti)
 val_acc = evaluate(model, test_dataloader, criterion, len(test_dataloader.dataset))
 print(f"[Epoch: {i:2d}], [Train Acc: {train_acc:3.4f}], [Val Acc: {val_acc:3.4f}]")

[Epoch:  0], [Train Acc: 0.0050], [Val Acc: 0.0100]
[Epoch:  1], [Train Acc: 0.0117], [Val Acc: 0.0100]
[Epoch:  2], [Train Acc: 0.0100], [Val Acc: 0.0100]
[Epoch:  3], [Train Acc: 0.0050], [Val Acc: 0.0100]
[Epoch:  4], [Train Acc: 0.0100], [Val Acc: 0.0100]
[Epoch:  5], [Train Acc: 0.0033], [Val Acc: 0.0100]
[Epoch:  6], [Train Acc: 0.0050], [Val Acc: 0.0200]
[Epoch:  7], [Train Acc: 0.0133], [Val Acc: 0.0400]
[Epoch:  8], [Train Acc: 0.0217], [Val Acc: 0.0400]
[Epoch:  9], [Train Acc: 0.0083], [Val Acc: 0.0400]
[Epoch: 10], [Train Acc: 0.0133], [Val Acc: 0.0500]
[Epoch: 11], [Train Acc: 0.0200], [Val Acc: 0.0500]
[Epoch: 12], [Train Acc: 0.0333], [Val Acc: 0.0600]
[Epoch: 13], [Train Acc: 0.0367], [Val Acc: 0.0700]
[Epoch: 14], [Train Acc: 0.0250], [Val Acc: 0.0800]
[Epoch: 15], [Train Acc: 0.0267], [Val Acc: 0.0800]
[Epoch: 16], [Train Acc: 0.0333], [Val Acc: 0.1000]
[Epoch: 17], [Train Acc: 0.0317], [Val Acc: 0.1300]
[Epoch: 18], [Train Acc: 0.0433], [Val Acc: 0.1300]
[Epoch: 19],